# LangChain 0.3 消息处理详解


LangChain 中的消息类型是与大语言模型交互的基础组件。消息类型提供了统一的格式，让你可以与不同的聊天模型无缝交互。

## 主要消息类型

LangChain 提供了几种核心消息类型：

1. **HumanMessage** - 对应用户输入
2. **AIMessage** - 对应AI助手回复
3. **SystemMessage** - 对应系统指令
4. **ToolMessage** - 对应工具调用结果
5. **FunctionMessage** - 对应函数调用结果（旧版API）


## 基础消息使用示例

In [1]:
from langchain_core.messages import (
    HumanMessage,
    AIMessage,
    SystemMessage
)
from langchain_ollama import ChatOllama

def basic_messages_demo():
    """基础消息类型示例"""

    # 初始化模型
    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建消息列表
    messages = [
        SystemMessage(content="你是一个专业的翻译助手"),
        HumanMessage(content="请将'Hello World'翻译成中文"),
    ]

    # 调用模型
    response = model.invoke(messages)
    print(f"翻译结果: {response.content}")
    #
    # 添加历史消息并继续对话
    messages.append(response)  # 添加AI回复到历史
    messages.append(HumanMessage(content="再翻译'Good morning'"))

    # 再次调用模型
    response = model.invoke(messages)
    print(f"第二次翻译: {response.content}")

if __name__ == "__main__":
    basic_messages_demo()

翻译结果: "Hello World" 翻译成中文是 "你好，世界"。不过，“Hello World”在编程和开发领域通常被用作一个示例字符串或程序的一部分，用来说明基本的语法或功能。如果你需要的是这句话的意思，则可以理解为“你好，世界”。
第二次翻译: 'Good morning' 翻译成中文是 '早上好'。



## 带额外参数的消息

In [4]:


# 消息可以包含额外的参数，用于传递更多信息：

from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama

def messages_with_kwargs_demo():
    """带额外参数的消息示例"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建带额外参数的消息
    detailed_message = HumanMessage(
        content="分析这段代码",
        additional_kwargs={
            "code": "def hello(): print('Hello')",
            "language": "python"
        }
    )

    # 调用模型
    response = model.invoke([detailed_message])
    print(f"代码分析: {response.content}")

if __name__ == "__main__":
    messages_with_kwargs_demo()


代码分析: 请提供你想要我进行分析的代码片段。


## 消息类型与对话记忆

In [ ]:
## 消息类型与对话记忆

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def message_memory_demo():
    """消息与对话记忆示例"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建聊天历史
    chat_history = []

    # 创建带消息占位符的提示模板
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="你是一个友好的AI助手"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ])

    # 第一轮对话
    user_input = "你好，我叫张三"
    messages = prompt.format_messages(chat_history=chat_history, input=user_input)
    response = model.invoke(messages)

    # 更新聊天历史
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response.content))

    print(f"用户: {user_input}")
    print(f"AI: {response.content}")

    # 第二轮对话
    user_input = "你还记得我的名字吗？"
    messages = prompt.format_messages(chat_history=chat_history, input=user_input)
    response = model.invoke(messages)

    print(f"用户: {user_input}")
    print(f"AI: {response.content}")

if __name__ == "__main__":
    message_memory_demo()


## 工具消息示例

In [5]:

## 工具消息示例


from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_community.chat_models import ChatOllama
import datetime

def tool_messages_demo():
    """工具消息示例"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 模拟工具调用结果
    def get_current_time():
        return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 创建消息列表
    messages = [
        SystemMessage(content="你是一个有用的AI助手"),
        HumanMessage(content="现在几点了？"),
        # 模拟工具调用并返回结果
        ToolMessage(
            content=get_current_time(),
            tool_call_id="time_tool_1",
            name="get_current_time"
        )
    ]

    # 调用模型
    response = model.invoke(messages)
    print(f"AI回复: {response.content}")

if __name__ == "__main__":
    tool_messages_demo()


ValueError: Received unsupported message type for Ollama.


## 多模态消息示例

In [ ]:

from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama

def multimodal_messages_demo():
    """多模态消息示例 (需要支持多模态的模型)"""

    # 注意：需要使用支持多模态的模型，如 llava 或 qwen-vl
    model = ChatOllama(
        base_url="http://localhost:11434",
        model="llava:7b-v1.6-mistral-q4_0"  # 确保已下载此模型
    )

    # 创建带图像URL的消息
    # 注意：实际使用时需要替换为真实可访问的图像URL
    image_message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": "这张图片里有什么？"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://example.com/image.jpg"  # 替换为实际图像URL
                }
            }
        ]
    )

    # 如果有支持多模态的模型，可以取消下面的注释
    # response = model.invoke([image_message])
    # print(f"图像分析: {response.content}")

    print("注意：多模态消息需要支持多模态的模型，如llava或qwen-vl")

if __name__ == "__main__":
    # 取消注释以运行多模态示例
    # multimodal_messages_demo()
    pass


## 消息流式处理

In [ ]:

from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama

def streaming_messages_demo():
    """消息流式处理示例"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建消息
    messages = [HumanMessage(content="写一首关于人工智能的短诗")]

    # 流式调用
    print("开始流式生成...")
    for chunk in model.stream(messages):
        # 打印每个生成的块
        print(chunk.content, end="", flush=True)
    print("\n流式生成完成")

if __name__ == "__main__":
    streaming_messages_demo()



## 消息与LangGraph结合

LangGraph是LangChain 0.3推荐的对话管理方式，可以更好地处理消息和状态：

In [ ]:

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, MessagesState
from langgraph.graph import START, END
from langgraph.checkpoint.memory import MemorySaver
import uuid

def langgraph_messages_demo():
    """LangGraph消息处理示例"""

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 定义模型调用函数
    def call_model(state: MessagesState):
        """调用模型并返回响应"""
        system_msg = SystemMessage(content="你是一个友好的AI助手，能够记住对话历史。")
        messages = [system_msg] + state["messages"]
        response = model.invoke(messages)
        return {"messages": [response]}

    # 创建状态图
    builder = StateGraph(MessagesState)
    builder.add_node("call_model", call_model)
    builder.add_edge(START, "call_model")
    builder.add_edge("call_model", END)

    # 编译图并添加内存检查点
    memory = MemorySaver()
    graph = builder.compile(checkpointer=memory)

    # 创建会话配置
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}

    # 第一轮对话
    print("\n第1轮对话:")
    user_input = "你好，我叫张三，是一名程序员"
    print(f"用户: {user_input}")

    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config
    )
    print(f"AI: {result['messages'][-1].content}")

    # 第二轮对话
    print("\n第2轮对话:")
    user_input = "你还记得我是做什么的吗？"
    print(f"用户: {user_input}")

    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config
    )
    print(f"AI: {result['messages'][-1].content}")

if __name__ == "__main__":
    langgraph_messages_demo()



## 总结

LangChain 0.3 的消息处理系统提供了：

1. **统一接口** - 不同模型提供商使用相同的消息格式
2. **丰富的消息类型** - 支持人类、AI、系统、工具等多种消息类型
3. **额外参数支持** - 可以添加额外信息到消息中
4. **流式处理** - 支持实时生成和显示
5. **与LangGraph集成** - 更好地管理对话状态和记忆

通过这些消息类型，你可以构建从简单对话到复杂多轮交互的各种应用。
